In [1]:
# o primeiro passo é importar as bibliotecas que vamos utilizar

from itertools import product # importando da biblioteca itertools o módulo produto
from math import sqrt # importando da biblioteca math o módulo raiz quadrada

import gurobipy as gp    #importando a biblioteca gurobi
from gurobipy import GRB #importando o módulo GRB

In [2]:
# em seguida inserimos os parâmetros do problema

clientes_coordenadas = [(0,1.5),(2.5,1.2)] # latitude e longitude
armazens_coordenadas = [(0,0),(0,1),(0,2),(1,0),(1,1),(1,2),(2,0),(2,1),(2,2)] # latitude e longitude
custo_instalacao = [3,2,3,1,3,3,4,3,2] # custo em milhões de reais
custo_km_percorrido = 3 # custo em milhões de reais

In [3]:
# agora vamos utilizar a função 'def' para definirmos as distância euclidiana entre 
# os armazens e os clientes

def computar_distancia(loc_cliente1,loc_cliente2):
    dx = loc_cliente1[0] - loc_cliente2[0]
    dy = loc_cliente1[1] - loc_cliente2[1]
    return sqrt(dx*dx + dy*dy)      

# equação para obtermos a distância em um espaço euclidiano bidimensional

In [4]:
# agora criamos objetos para computar os parâmetros do problema 

num_armazens = len(armazens_coordenadas)   #retorna o número de armazens
num_clientes = len(clientes_coordenadas)  #retorna o número de clientes
produto_cartesiano = list(product(range(num_clientes),(range(num_armazens))))

# em seguida criamos um objeto para calcular os custos de transporte
custo_transporte = {(c,a): custo_km_percorrido*computar_distancia(clientes_coordenadas[c], armazens_coordenadas[a]) for c, a in produto_cartesiano}


In [5]:
# agora vamos formular um modelo inteiro misto (MIP)

m = gp.Model('localização_armazens') #criando o modelo

selecionar = m.addVars(num_armazens, vtype=GRB.BINARY, name='selecionar') #inserindo a variável decisória binária armazem ativo/inativo no modelo
alocar = m.addVars(produto_cartesiano, ub=1, vtype=GRB.CONTINUOUS, name='alocar') #inserindo a variável decisória contínua fluxo no modelo

m.addConstrs((alocar[(c,a)] <= selecionar[a] for c,a in produto_cartesiano), name='preparar para envio') #adicionando restrição de capacidade onde só enviamos de armazens ativos
m.addConstrs((gp.quicksum(alocar[(c,a)] for a in range(num_armazens)) == 1 for c in range(num_clientes)), name='demanda') #adicionado restrição de demanda onde a soma das frações é igual a 1

m.setObjective(selecionar.prod(custo_instalacao)+alocar.prod(custo_transporte), GRB.MINIMIZE) #inserindo a função objetivo

m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 20 rows, 27 columns and 54 nonzeros
Model fingerprint: 0x3f7f780f
Variable types: 18 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 20 rows, 27 columns, 54 nonzeros
Variable types: 18 continuous, 9 integer (9 binary)
Found heuristic solution: objective 8.1155494

Root relaxation: cutoff, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0         8.11555    8.11555  0.00%     -    0s

Explored 1 nodes (7 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 8 (of 8 availabl

**ANÁLISE DA SOLUÇÃO** 

O resultado da solução mostra que o custo mínimo total é de 8,11 milhões de reais

In [6]:
# em seguida vamos mostrar a localização dos armazens selecionados utilizando um algoritmo

for facility in selecionar.keys():
    if(abs(selecionar[facility].x) > 1e-6):
        print(f'\n Construa um armazem na localização {facility + 1}.')
    


 Construa um armazem na localização 2.

 Construa um armazem na localização 8.


In [7]:
# agora vamos mostrar a capacidade utilizada dos armazens

for customer, facility in alocar.keys():
    if(abs(alocar[customer, facility].x) > 1e-6):
        print(f'\n Supermercado {customer +1} recebe {round(100*alocar[customer, facility].x, 2)} % de sua demanda do armazem {facility +1}.')
    


 Supermercado 1 recebe 100.0 % de sua demanda do armazem 2.

 Supermercado 2 recebe 100.0 % de sua demanda do armazem 8.
